In [24]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import numpy as np
from sklearn.model_selection import train_test_split
import tokenization
import torch
import transformers as ppb

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from chart_studio.plotly import plot, iplot


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier


import matplotlib.pyplot as plt   
from keras.models import Sequential  
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

from keras.regularizers import l2

from keras.layers import Dense, Embedding, SpatialDropout1D, Dropout, Conv1D
from keras.layers import Input, Add, concatenate, Dense, Activation, BatchNormalization, Dropout, Flatten
from keras.layers import LSTM, Bidirectional, GRU, GlobalAveragePooling1D, MaxPooling1D, GlobalMaxPooling1D


from sklearn.preprocessing import StandardScaler, MinMaxScaler



In [3]:
def resultados(pred, test_df):
    res_df=pd.DataFrame(test_df['id'])
    res_df['target']=pred
    res_df.to_csv('data/submission.csv', index=False)

In [4]:
train_df = pd.read_csv('data/train_fttd.csv')
test_df = pd.read_csv('data/test_fttd.csv')

In [5]:
DEVICE = torch.device("cpu")
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
model.to(DEVICE)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_fe

Aca tuve que partir los datos porque al consumir mucha memoria crasheaba. SIGKILL 137

In [6]:
frames_train = [train_df[0:3000], train_df[3000:6000], train_df[6000:7613]]
frames_test = [test_df[0:1500],test_df[1500:3000], test_df[3000:3263]]

In [7]:
features = []
def get_features(frames):
    features = []
    for df in frames:
        tokenized = df['text_clean'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
        max_len = 0
        for i in tokenized.values:
            if len(i) > max_len:
                max_len = len(i)
        padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

        attention_mask = np.where(padded != 0, 1, 0)
        attention_mask.shape
        input_ids = torch.tensor(padded).to(DEVICE)  
        attention_mask = torch.tensor(attention_mask).to(DEVICE)
        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask)
        features.append(last_hidden_states[0][:,0,:].numpy())
    return features

In [8]:
print('Matriz de datos de entrenamiento...')
train_features = get_features(frames_train)
train_features = np.vstack((train_features[0], train_features[1], train_features[2]))

Matriz de datos de entrenamiento...


# Entreno con LR

In [15]:
#Split
y = train_df['target']
X_train, X_test, y_train, y_test = train_test_split(train_features, y, test_size = 0.25, random_state = 12)


In [21]:
#Busco parámetros buenos
scores = []
cs = []
for i in np.arange(0.01, 1, 0.01):
    LR = LogisticRegression(penalty='l2',  C=i, max_iter=100)
    LR.fit(X_train, y_train)
    scores.append(LR.score(X_test, y_test))
    cs.append(i)

GRAFICO

In [23]:
#Grafico
fig = go.Figure()
fig.add_trace(go.Scatter(x=cs, y=scores,
                    mode='lines'))
fig.update_layout(title_text='Logistic Regression: Exactitud en función de C',
                  xaxis_title='C',
                  yaxis_title='Accuracy',
                  showlegend=False,
                  template="plotly")
fig.show()

# CROSS VAL

In [17]:
X_train, X_test, y_train, y_test = train_test_split(train_features, y, test_size = 0.25, random_state = 7506)
LR = LogisticRegression(penalty='l2',  C=0.45, max_iter=100)
scores = cross_val_score(LR, X_train, y_train, cv=5, scoring='accuracy')
print(scores)

[0.8117338  0.82224168 0.8117338  0.81698774 0.80718668]


# Submit

In [11]:
print('Matriz de datos de prueba...')
test_features = get_features(frames_test)
test_features = np.vstack((test_features[0], test_features[1], test_features[2]))


Matriz de datos de prueba...


In [22]:
#Target
y_sub = train_df['target']


#Entrenamiento con todo el set de datos y submit
LR = LogisticRegression(penalty='l2',  C=0.45, max_iter=100)
LR.fit(train_features, y_sub)

#Salida
y_pred = LR.predict(test_features)
resultados(y_pred, test_df)